In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.dummy import DummyClassifier
import sklearn.metrics as skm
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

url = 'https://raw.githubusercontent.com/mirea-aie-2025/aie-course-meta/refs/heads/main/seminars/S05/S05-hw-dataset.csv'
df = pd.read_csv(url)
print(df.head())
print('=' * 52)
print(df.info())
print('=' * 52)
print(df.describe())
print('=' * 52)
print(df.value_counts(normalize=True))
print('=' * 52)
X = df.drop('client_id', axis=1)
X = X.drop('default', axis=1)
y = df.default
print(abs(1 -len(df[df.default == 1])/len(df)))
# Датасет состоит из 3000 объектов и имеет 17 признаков, и разделён на два класса по таргету 'default' в соотношении 59 на 41

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

bl = DummyClassifier(strategy='most_frequent', random_state=42)
bl.fit(X_train, y_train)
y_proba = bl.predict_proba(X_test)[:, 1]
y_pred = bl.predict(X_test)
print("Accuracy :", skm.accuracy_score(y_test, y_pred))
print("ROC-AUC  :", skm.roc_auc_score(y_test, y_proba))
# Бейзлайн классифицировал датасет на две группы по таргету 'default'. Точность такой модели классификации полностью соответствует соотношению классов полученному в предыдущем пункте.
# Точность этой модели соответствует нижней планке для моделей классификации. Если точность итоговой модели будет ниже этой (0.59), то это будет означать её абсолютную неэффективность.

print('=' * 52)
lr = Pipeline(steps=[('skelter', StandardScaler()), ("logreg", LogisticRegression(penalty='l2' , C=1.0, solver='liblinear', random_state=42))])
lr.fit(X_train, y_train)
y_lr_pred = lr.predict(X_test)
y_lr_proba = lr.predict_proba(X_test)[:, 1]
print("Accuracy :", skm.accuracy_score(y_test, y_lr_pred))
print("ROC-AUC  :", skm.roc_auc_score(y_test, y_lr_proba))
print('=' * 52)
gs = GridSearchCV(
    estimator=lr,
    param_grid={"logreg__C": [0.01, 0.1, 1.0, 10.0, 100.0],},
    scoring="roc_auc",
    cv=5,
    n_jobs=-1,
    verbose=1,
)
gs.fit(X_train, y_train)
print("Лучшие параметры:", gs.best_params_)
print("Лучший ROC-AUC (по CV):", gs.best_score_)
print('=' * 52)
bm = gs.best_estimator_
y_lr_best_pred = bm.predict(X_test)
y_lr_best_proba = bm.predict_proba(X_test)[:, 1]
print("Accuracy :", skm.accuracy_score(y_test, y_lr_best_pred))
print("ROC-AUC  :", skm.roc_auc_score(y_test, y_lr_best_proba))
skm.RocCurveDisplay.from_predictions(
    y_test,
    y_lr_best_proba,
)
plt.savefig('figures/f1')
plt.show()
skm.PrecisionRecallDisplay.from_predictions(
    y_test,
    y_lr_best_proba,
)
plt.savefig('figures/f2')
plt.show()
# Логическая регрессия показала значительно лучший результат, чем бейзлайн (Accuracy: 0.8 против 0.59 и ROC-AUC: 0.87 против 0.5)
# Подобная разница происходит из-за того, что бейзлайн грубо оценивает принадлежность всех объектов к одному классу (как если бы мы записали все объекты в самый многочисленный класс).
# В свою очередь линейная регрессия является полноценной моделью классификации, показывающей значительно лучший результат.
# Однако подбор лучших параметров регуляризации для C (базово был задан С=1.0, а лучший С=10.0) не дал значительных результатов (отличае лишь в сотых долях процента).
# Модель логической регрессии кажется разумной для этой задачи, ибо она показывает довольно высокую точность (0.8) на заданом датасете и ROC-AUC (0.88).